<a href="https://https://colab.research.google.com/github/TestoryTech/examples/main/TestingMagentoWithTestory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

## Install the testory tool and define an alias for it

In [ ]:
!wget -nc https://github.com/TestoryTech/examples/releases/download/0.6.5/testory.sh
!wget -nc https://github.com/TestoryTech/examples/releases/download/0.6.5/TestoryTool-101.uber.jar
!chmod +x testory.sh
%alias testory /content/testory.sh

## Install Selenium and run a standalone server

In [ ]:
!wget -nc https://github.com/SeleniumHQ/selenium/releases/download/selenium-4.1.0/selenium-server-4.1.2.jar
!apt-get update
!apt-get install chromium-chromedriver

In [ ]:
%%script bash --bg
java -jar /content/selenium-server-4.1.2.jar standalone --port 4444 --override-max-sessions true --max-sessions 40  --log /content/selenium.log

# Test model

## Business Logic

### Metadata

Testory allows to define constants that the test stories can use to define generic test cases. The box below demonstrates how we deifine some constants that we later use in our test scripts. These constants can depent on evironment variables, as demonstra by the URL variable. In our example, we use an environment variable to synchronize the test model with the setup script `empty_cart.py` shown below.

In [ ]:
# A local Magento sever. 
# Useful when connecting to a colab runtime that runs the magento store.
# %env URL http://localhost 

# A public Magebto server.
# Note that the server reboots at round hours and is not available for few minutes at these times.
%env URL = https://magento2-demo.magebit.com/

In [ ]:
%%writefile _constants.story.js 
//%%js
// The address of the store that we wish to test
const URL = getenv("URL")

// The number of concurrent purchase sessions
const NUM_OF_SESSIONS = 2

// A list of products that we can add to the cart
const products = {
    'Stark Fundamental Hoodie': {category: 'Men', subCategory: 'Tops', subSubCategory: 'Hoodies & Sweatshirts', options: ['M', 'Blue'], product: 'Stark Fundamental Hoodie'},
    'Atomic Endurance Running Tee (V-neck)': {category: 'Men', subCategory: 'Tops', subSubCategory: 'Tees', options: ['S', 'Yellow'], product: 'Atomic Endurance Running Tee (V-neck)'},
    'Meteor Workout Short': {category: 'Men', subCategory: 'Bottoms', subSubCategory: 'Shorts', options: ['34', 'Blue'], product: 'Meteor Workout Short'},
    'Layla Tee': {category: 'Women', subCategory: 'Tops', subSubCategory: 'Tees', options: ['S', 'Green'], product: 'Layla Tee'},
    'Bella Tank': {category: 'Women', subCategory: 'Tops', subSubCategory: 'Bras & Tanks', options: ['S', 'Orange'], product: 'Bella Tank'},
    'Inez Full Zip Jacket': {category: 'Women', subCategory: 'Tops', subSubCategory: 'Jackets', options: ['Orange', 'L'], product: 'Inez Full Zip Jacket'}
}


// The names of the products that we can add to the cart
const productNames = Object.keys(products)

// The credentials of the user that we want to test with
const username = 'roni_cost@example.com'
const password = 'roni_cost3@example.com'
const expectedWelcome =  'Welcome, Veronica Costello!'

### Add to cart stories
In the code below we define our first test script. The stcript uses the constants `NUM_OF_SESSIONS`, `productNames`, and `products` to define stories that login, choose a product, and add it to the cart. Each story is defined to run in a separate session. See [checkout.story.js](https://colab.research.google.com/drive/15WASmf07bnamFugxRxHrPYZKVY3tpSie/content/checkout.story.js).

In [ ]:
%%writefile add_to_cart.story.js 
//%%js
for (let i = 0; i < NUM_OF_SESSIONS; i++) {
    story("AddToCartStory" + i, function () {
        // Start a new browser session on the store URL
        with (new SeleniumSession().start(URL)) {
            // Log-in as a regular user
            login({username: username, password: password, expectedWelcome: expectedWelcome})

            // Choose a random product
            let prod = choose(productNames)

            // Add the chosen product to the shopping cart
            addToCart(products[prod])
        }
    })
}

### Check out story

Our second script, shown below, defines a story for checking out product from the store. In this story, our user loggs in and checks out all the item in the shopping cart. While doing so, it listens to all `AddToCart` events and maintains two lists: a list of the product put in the cart and a list of products that are not in the cart. At the end, the story uses this information to tell the `checkOut` event to verify the shopping cart both positively and negatively.

In [ ]:
%%writefile checkout.story.js 
//%%js
story("CheckOutStory", function () {
    let incart = []

    let AddToCartListener = {
        eventset: Any("AddToCart"),
        callback: e => {incart.push(e.data.product)}
    }

    waitFor(AddToCartListener, function () {
        with (new SeleniumSession().start(URL)) {
            // To bypass Magento's bug...
            waitFor(Any(/Click \[session=AddToCartStory\d xpath=\/\/button\[@id='product-addtocart-button']\/span]/))

            // Login to the store
            login({username: 'roni_cost@example.com', password: 'roni_cost3@example.com', expectedWelcome: 'Welcome, Veronica Costello!'})

            // Checkout and verify existence/nonexistence of items in the cart
            checkOut({
                shippingMethod: 'Fixed',
                verifyItems: incart,
            })
        }
    })
})

### Add before checkout story

Finally, we add a story that tells testory that we wish to finish all `AddToCart` stories before procedding to `CheckOut`.

In [ ]:
%%writefile add_before_checkout.story.js 
//%%js
story("AddBeforeCheckoutStory", function () {
    block(Any("CheckOut"), function () {
        for (let i = 0; i < NUM_OF_SESSIONS; i++)
            waitFor(Any(/EndStory-AddToCartStory/))
    })
})


## Infrastructure

### Event Definition

The stories above use events such as `AddToCart` and `Login` that are specific to testing the Magento store. This is an example of how testory allows for separation of concerns. The business logic is defined above while the delails of the events are encaspulated in an event definition file.

Expand this section to see the event definition file. Each event is defined using the `define_event` function whose first parameter is the name of the event and second parameter is a callback function that translates the event to Selenium commands. The callback function takes two parameters: a `session` in which the event is invoked and the `event` itself (the `data` field of the BEvent object).


In [ ]:
%%writefile events.js 
//%%js
/* @Testory summon selenium */

/***********************************************************************************
 * Login to the store as a regular user.
 *
 * Parameters:
 *   username: string - The user that logs in
 *   password: string - The password of that user
 ************************************************************************************/
define_event("Login", function(session, event) {
    with(session) {
        click("//a[contains(text(),'Sign In')]");
        writeText('//input[@id="email"]', event.username);
        writeText('//input[@id="pass"]', event.password);
        click('//button[@id="send2"]');

        if (event.expectedWelcome)
            waitForVisibility("//span[text()='" + event.expectedWelcome + "']", 10)
    }
})


/***********************************************************************************
 * Login to the store as an admin user.
 *
 * Parameters:
 *   username: string - The user that logs in
 *   password: string - The password of that user
 ************************************************************************************/
define_event("AdminLogin", function(session, event) {
    with(session) {
        writeText('//input[@id="username"]', event.username);
        writeText('//input[@id="login"]', event.password);
        click("//span[text()='Sign in']");
    }
});

/***********************************************************************************
 * Logout a regular user.
 *
 ************************************************************************************/
define_event("Logout", function(session, event) {
    with(session) {
        click("//span[@class='customer-name']//button");
        click("//a[normalize-space()='Sign Out']");
    }
});

/***********************************************************************************
 * Register a  user.
 *
 * Parameters:
 *   s: string              - The name of the session in which we want this event to take place
 *   firsntame : string     - The name of the new user
 *   lastname : string      - The surname of the new user
 *   email_address : string - An email address for the user. Must be unique.
 *   password : string      - Password for the new user.
 ************************************************************************************/
define_event("Register", function(session, event) {
    with(session) {
        click("//a[@href='http://localhost/customer/account/create/']");
        writeText('//input[@id="firstname"]', event.firstname);
        writeText('//input[@id="lastname"]', event.lastname);
        writeText('//input[@id="email_address"]', event.email_address);
        writeText('//input[@id="password"]', event.password);
        writeText('//input[@id="password-confirmation"]', event.password);
        click('//button[@type="submit" and contains(concat(" ",normalize-space(@class)," ")," action ") and contains(concat(" ",normalize-space(@class)," ")," submit ")]');
        assertText("//div[@data-ui-id='message-success']//div[1]", "Thank you for registering with Main Website Store.")
    }
});


/***********************************************************************************
 * Add an item to the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string                  - The name of the session in which we want this event to take place.
 *   category : string          - The category of the product that we want to add.
 *   subCategory : string       - The sub-category of the product that we want to add.
 *   product : string           - The  product that we want to add.
 *   options : array of strings - A list of options for the product.
 *   quantity: number, optional - The number of items to add.
 ************************************************************************************/
define_event("AddToCart", function(session, event) {
    with(session) {

        click("//span[text()='" + event.category + "']");
        click("(//span[text()='" + event.category + "'])/following::span[text()='" + event.subCategory + "']/following::a[text()[normalize-space()='" + event.subSubCategory + "']]");

        click("(//img[@alt='" + event.product + "'])[last()]");
        for (let opt of event.options) {
            // Click the options
            click("//div[@data-option-label='" + opt + "']");

            // Verify that it was selected
            waitForVisibility("//div[@data-option-label='" + opt + "' and contains(@class,'selected')]");
        }
        if (event.quantity) {
            writeText("//input[@title='Qty']", event.quantity, true);
        }
        click("//button[@id='product-addtocart-button']/span");
        waitForVisibility("//div[@data-ui-id='message-success']//div[1]", 15);
        assertText("//div[@data-ui-id='message-success']//div[1]", "You added " + event.product + " to your shopping cart.");
    }
});

/***********************************************************************************
 * Remove an item from the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string        - The name of the session in which we want this event to take place.
 *   product : string - The  product that we want to remove.
 ************************************************************************************/
define_event("RemoveFromCart", function(session, event) {
    with(session) {
        click("//a[@class='action showcart']");
        click("//a[text()[normalize-space()='" + event.product + "']]/following::a[@class='action delete']");
        click("//div[text()='Are you sure you would like to remove this item from the shopping cart?']/following::span[text()='OK']");
        waitForInvisibility("//div[contains(@class,'block block-minicart')]//img[@alt='" + event.product + "']");
        click("//button[@id='btn-minicart-close']");
    }
});

/***********************************************************************************
 * Check that a product exists in the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string -      - The name of the session in which we want this event to take place.
 *   product : string - The  product that we want to remove.
 ************************************************************************************/
define_event("CheckExistenceOfProductInCart", function(session, event) {
    with(session) {
        click("//a[@class='action showcart']");
        waitForVisibility("//div[contains(@class,'block block-minicart')]//img[@alt='" + event.product + "']");
        click("//button[@id='btn-minicart-close']");
    }
});

/***********************************************************************************
 * Check-out the items in the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string                                              - The name of the session in which we want this event to take place.
 *   verifyItems : array of strings, optional               - A list of items that we expect to see in the cart.
 *   verifyNonexistenceOfItems : array of strings, optional - A list of items that we expect not to see in the cart.
 *   shippingMethod : string, optional                      - The shopping method that we want to use for this order.
 ************************************************************************************/
define_event("CheckOut", function(session, event) {
    with(session) {
        click("//a[@class='action showcart']");
        click("//button[@title='Proceed to Checkout']");

        if (event.verifyItems || event.verifyNonexistenceOfItems) {
            waitForVisibility("//div[@class='block items-in-cart']//div", 20);
            click("//div[@class='block items-in-cart']//div");
        }

        if (event.verifyItems) {
            for (item of event.verifyItems) {
                waitForVisibility("//img[@alt='" + item + "']");
            }
        }

        if (event.verifyNonexistenceOfItems) {
            for (item of event.verifyNonexistenceOfItems) {
                waitForInvisibility("//img[@alt='" + item + "']", 5);
            }
        }

        if (event.shippingMethod) {
            waitForVisibility("//td[text()='" + event.shippingMethod + "']", 5);
            click("//td[text()='" + event.shippingMethod + "']");
        }

        click("//span[text()='Next']");

        if (event.verifyItems) {
            for (item of event.verifyItems) {
                waitForVisibility("//img[@alt='" + item + "']");
            }
        }
        if (event.verifyNonexistenceOfItems) {
            for (item of event.verifyNonexistenceOfItems) {
                waitForInvisibility("//img[@alt='" + item + "']", 5);
            }
        }

        waitForVisibility("//span[text()='Place Order']", 20);
        click("//span[text()='Place Order']");
        waitForVisibility("//p[text()='Your order number is: ']", 5);
        click("//span[text()='Continue Shopping']");
    }
}); 

### Setup script

Expand this section to see a setup script that we use in this example. This is a regular python script that uses Magento's REST interface to empty the shopping cart of the user whose actions we are simulating in our test. We will run this script using the `--before-test ./empty_cart.py`, as shown [below](https://colab.research.google.com/drive/15WASmf07bnamFugxRxHrPYZKVY3tpSie#scrollTo=Ad4AxpKhiMeM&line=1&uniqifier=1).

In [ ]:
%%writefile empty_cart.py 
#!/usr/bin/env python
import json
import os
from requests import post, get, delete


URL = os.getenv("URL")
CREDENTIALS = {'username': "roni_cost@example.com", 'password': "roni_cost3@example.com"}

r = post(f'{URL}/rest/default/V1/integration/customer/token', params=CREDENTIALS)
token = r.text[1:-1]
header = {'Authorization': f'Bearer {token}'}

r = get(f'{URL}/rest/default/V1/carts/mine', headers=header)
cart = json.loads(r.text)

if "items" in cart:
    for item in cart["items"]:
        delete(f'{URL}/rest/default/V1/carts/mine/items/{item["item_id"]}', headers=header)

In [ ]:
!chmod +x empty_cart.py

# Run


## Sampling

Testory various tools to generate tests from the model. In this demonstration, we just sample 10 random tests that satisfy the model. The following command deletes the file `10tests.json` and writes and ensemble of 10 random tests to it.  

In [ ]:
testory sample --delete --samples-file 10tests.json --sample-size=10  .

## Running the generated ensemble

To run the ensemble fron `10tests.json`, we use the following command. Note the use of the `--before-test ./empty_cart.py` flag to specify that we want to invoke `empty_cart.py` before each test. This is an example of using a setup script to initialize the state before each test. 

The script below should take some time to run, as it interracts with the online store. Note that the store is taken down every round our for five minutes, so expect error if you run the tests at these times.

In [ ]:
testory run --run-ensemble --ensemble-file 10tests.json --before-test ./empty_cart.py .

## Generating and viewing a report

Testory supports various types of reports. In this example, we use the command below to geberate a report that details the steps of the tests that we have executed.

In [ ]:
testory report TestLog .

The generated report can be displayed by clicking on the link below.

In [ ]:
from google.colab import output
get_ipython().system_raw('python3 -m http.server 8888 --directory /content/report&') 
output.serve_kernel_port_as_window(8888, path="extent/html/index.html")